In [30]:
#!pip install bertopic
#!pip install openpyxl

In [31]:
from pandas import read_excel
data = read_excel("sanitized_data.xlsx",sheet_name = 'sanitized_text')
data

,Unnamed: 0,tweet_id,original_tweets,company_name,grams_to_text
0,0,1057794087955427328,"Tomorrow, we walk out for real change. #google...",Google,"tomorrow, walk real change. #googlewalkout #go..."
1,1,1057804203895283712,"We, Google employees and contractors, will wal...",Google,"we, employees contractors, walkout november 1 ..."
2,2,1057826104965246976,Google employees and contractors will be leavi...,Google,employees contractors leaving flyers desks tom...
3,3,1057836978954256385,@sasha_feather @hypatiadotca Thank you so much...,Google,"thank reminder! we’re it, sorry lapse."
4,4,1057837349290287104,Text (1/2):\n1. An end to Forced Arbitration i...,Google,text (1/2): 1. end forced_arbitration cases ha...
...,...,...,...,...,...
3130,3167,1471934948336615430,"7/ As workers, we’re disturbed. “This report i...",Amazon,"7/ workers, we’re disturbed. “this report espe..."
3131,3168,1471934946625429510,6/ Maybe the scariest part about all of this i...,Amazon,6/ maybe scariest plans grow warehouse footpri...
3132,3169,1471934950073151496,8/ Thank you @kavehwaddell @ConsumerReports @g...,Amazon,8/ thank important investigative work. lai...
3133,3170,1472942398946910211,1/ I've worked for Apple since August 2015. \n...,Apple,1/ worked apple august 2015. harassment gaslig...


In [32]:
df_nan = data[data['grams_to_text'].isna()]
df_nan


,Unnamed: 0,tweet_id,original_tweets,company_name,grams_to_text


In [33]:
#Drop NaN rows
data=data.dropna()
data.rename( columns={'Unnamed: 0':'original_document_num'}, inplace=True )
data.shape

(3135, 5)

In [34]:
from bertopic import BERTopic
topic_model = BERTopic(embedding_model="all-MiniLM-L6-v2",nr_topics=20)

In [35]:
docs = data['grams_to_text'].values.tolist()
topics, probs = topic_model.fit_transform(docs)

In [36]:
"""
Note that in the above data frame, topic -1 denotes the topic consisting of outlier documents which are typically ignored due to terms having a relatively high prevalence 
across the whole corpus and thus, low specificity toward any cohesive theme or topic.
"""
df_topics = topic_model.get_topic_info()
df_topics.drop(df_topics.loc[df_topics['Topic']==-1].index, inplace=True) #Droping topic -1
df_topics
#len(df_topics.index)
#df_topic_name_number_mapping[df_topic_name_number_mapping.Topic == 0]

,Topic,Count,Name
1,0,204,0_ice_github_notechforice_githubdropice
2,1,112,1_ai_research_dr_gebru
3,2,111,2_climatestrike_fridaysforfuture_amznclimate_c...
4,3,110,3_googlewalkout_nyc_san_timesup
5,4,92,4_microsoft_mspac_employees_jedi
6,5,84,5_google_googlers_walkoutyearinreview_19
7,6,73,6_arbitration_forced_arbitration_endforcedarbi...
8,7,72,7_68_drawalinetableau_drew_line_when
9,8,70,8_apple_appletoo_discrimination_issues
10,9,67,9_warehouse_workers_amazon_protectamazonworkers


In [37]:
topic_model.get_topic(-1) 
##You can manually merge topics that u think are similar
## https://maartengr.github.io/BERTopic/getting_started/topicreduction/topicreduction.html


[('workers', 0.02744899410548634),
 ('climate', 0.023013173113778863),
 ('employees', 0.017397734776191533),
 ('company', 0.015269705386884954),
 ('tech', 0.014935612497779052),
 ('work', 0.013838854580359217),
 ('people', 0.01305849937889139),
 ('stand', 0.012976127509381883),
 ('amazon', 0.012914955090422562),
 ('change', 0.012808494501585957)]

In [38]:
topic_model.get_document_info(docs)

,Document,Topic,Name,Top_n_words,Probability,Representative_document
0,"tomorrow, walk real change. #googlewalkout #go...",-1,-1_workers_climate_employees_company,workers - climate - employees - company - tech...,0.000000,False
1,"we, employees contractors, walkout november 1 ...",18,18_walkout_googlewalkout_employees_protest,walkout - googlewalkout - employees - protest ...,0.918179,False
2,employees contractors leaving flyers desks tom...,18,18_walkout_googlewalkout_employees_protest,walkout - googlewalkout - employees - protest ...,0.918179,False
3,"thank reminder! we’re it, sorry lapse.",14,14_thank_sorry_dms_you,thank - sorry - dms - you - cc - dm - password...,0.700952,False
4,text (1/2): 1. end forced_arbitration cases ha...,6,6_arbitration_forced_arbitration_endforcedarbi...,arbitration - forced_arbitration - endforcedar...,0.249781,False
...,...,...,...,...,...,...
3130,"7/ workers, we’re disturbed. “this report espe...",9,9_warehouse_workers_amazon_protectamazonworkers,warehouse - workers - amazon - protectamazonwo...,0.585079,False
3131,6/ maybe scariest plans grow warehouse footpri...,15,15_pollution_communities_warehouses_air,pollution - communities - warehouses - air - a...,0.835226,False
3132,8/ thank important investigative work. lai...,15,15_pollution_communities_warehouses_air,pollution - communities - warehouses - air - a...,0.364085,False
3133,1/ worked apple august 2015. harassment gaslig...,-1,-1_workers_climate_employees_company,workers - climate - employees - company - tech...,0.000000,False


In [39]:
#topic_model.visualize_topics()

In [40]:
#topic_model.visualize_barchart()

In [41]:
#import pprint 
#pprint.pprint(topic_model.get_representative_docs(topic=2))


In [42]:
#import pandas as pd
#df_topic_document = pd.DataFrame({'topic': topics, 'document': docs})

In [43]:
#topic_0 = df_topic_document[df_topic_document.topic == 0]
#topic_0

In [44]:
topic_distr, _ = topic_model.approximate_distribution(docs)

In [45]:
topic_distr[1]

array([0.        , 0.        , 0.        , 0.10490462, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.1095015 , 0.04045579, 0.        ,
       0.        , 0.        , 0.        , 0.74513809, 0.        ])

In [46]:
#topic_model.visualize_distribution(topic_distr[1])

In [47]:
import pandas as pd
df_output = pd.DataFrame({'Tweet_id': data['tweet_id'],'tweet':data['original_tweets'],'company': data['company_name']})
df_topics

#Adding topics as columns headers to the output dataframe
for index,row in df_topics.iterrows():
    topic_name = row['Name']
    df_output[topic_name] = 0

df_output.head(5)

,Tweet_id,tweet,company,0_ice_github_notechforice_githubdropice,1_ai_research_dr_gebru,2_climatestrike_fridaysforfuture_amznclimate_climate,3_googlewalkout_nyc_san_timesup,4_microsoft_mspac_employees_jedi,5_google_googlers_walkoutyearinreview_19,6_arbitration_forced_arbitration_endforcedarbitration_fairact,...,10_digest__appletoo_live_stories_compiled,11_leadership_shareholder_18_we,12_wayfair_wayfairwalkout_camps_beds,13_tvcs_tvc_full_pittsburgh,14_thank_sorry_dms_you,15_pollution_communities_warehouses_air,16_black_women_woman_color,17_tech_ethical_activist_public,18_walkout_googlewalkout_employees_protest,19_googlepayoutsforall_135m_000_fund
0,1057794087955427328,"Tomorrow, we walk out for real change. #google...",Google,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1057804203895283712,"We, Google employees and contractors, will wal...",Google,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1057826104965246976,Google employees and contractors will be leavi...,Google,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1057836978954256385,@sasha_feather @hypatiadotca Thank you so much...,Google,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1057837349290287104,Text (1/2):\n1. An end to Forced Arbitration i...,Google,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [48]:
number_of_docs = len(docs)
number_of_topics = len(df_topics.index)
#print(number_of_topics)
for doc_num in range(number_of_docs):
    topics = topic_distr[doc_num]
    #print(len(topics))
    col_index = 3
    #print (type(doc_num))
    for topicNum in range(number_of_topics):
        #print (type(topicNum))
        #print(topics[topicNum])
        df_output.iloc[[doc_num],[col_index]] = topics[topicNum]
        col_index = col_index + 1

df_output.head(5)

In [49]:
df_output.head(100)

,Tweet_id,tweet,company,0_ice_github_notechforice_githubdropice,1_ai_research_dr_gebru,2_climatestrike_fridaysforfuture_amznclimate_climate,3_googlewalkout_nyc_san_timesup,4_microsoft_mspac_employees_jedi,5_google_googlers_walkoutyearinreview_19,6_arbitration_forced_arbitration_endforcedarbitration_fairact,...,10_digest__appletoo_live_stories_compiled,11_leadership_shareholder_18_we,12_wayfair_wayfairwalkout_camps_beds,13_tvcs_tvc_full_pittsburgh,14_thank_sorry_dms_you,15_pollution_communities_warehouses_air,16_black_women_woman_color,17_tech_ethical_activist_public,18_walkout_googlewalkout_employees_protest,19_googlepayoutsforall_135m_000_fund
0,1057794087955427328,"Tomorrow, we walk out for real change. #google...",Google,0.0,0.0,0.0,0.471798,0.0,0.151237,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.376966,0.0
1,1057804203895283712,"We, Google employees and contractors, will wal...",Google,0.0,0.0,0.0,0.104905,0.0,0.000000,0.000000,...,0.0,0.000000,0.109502,0.040456,0.000000,0.0,0.0,0.0,0.745138,0.0
2,1057826104965246976,Google employees and contractors will be leavi...,Google,0.0,0.0,0.0,0.245235,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.754765,0.0
3,1057836978954256385,@sasha_feather @hypatiadotca Thank you so much...,Google,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,...,0.0,0.187405,0.000000,0.000000,0.812595,0.0,0.0,0.0,0.000000,0.0
4,1057837349290287104,Text (1/2):\n1. An end to Forced Arbitration i...,Google,0.0,0.0,0.0,0.000000,0.0,0.000000,0.619538,...,0.0,0.000000,0.066489,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1058059546332139520,greatest sign or greatest sign #googlewalkout ...,Google,0.0,0.0,0.0,1.000000,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0
96,1058059737932140545,"We see you, Google Kirkland! #GoogleWalkout ht...",Google,0.0,0.0,0.0,1.000000,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0
97,1058059938344456193,"Thanks for braving the cold, Google Seattle! #...",Google,0.0,0.0,0.0,1.000000,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0
98,1058060088240496643,"Google San Francisco, great turnout! #GoogleWa...",Google,0.0,0.0,0.0,1.000000,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0


In [50]:
df_output['Tweet_id'] = df_output['Tweet_id'].apply(str)

# create excel writer object
writer1 = pd.ExcelWriter('bert_topic_modelling.xlsx')
# write dataframe to excel
df_output.to_excel(writer1, sheet_name = 'topic_modelling', engine="xlsxwriter")

# save the excel
writer1.save()
#writer.close()
print('DataFrame is written successfully to Excel File.')

DataFrame is written successfully to Excel File.
